In [1]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)

In [ ]:
# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines(
    "BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC"
)

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines(
    "ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018"
)

# fetch weekly klines since it listed
klines = client.get_historical_klines(
    "NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017"
)

# [
#     1499040000000,      // 0 Open time
#     "0.01634790",       // 1 Open
#     "0.80000000",       // 2 High
#     "0.01575800",       // 3 Low
#     "0.01577100",       // 4 Close
#     "148976.11427815",  // 5 Volume
#     1499644799999,      // 6 Close time
#     "2434.19055334",    // 7 Quote asset volume
#     308,                // 8 Number of trades
#     "1756.87402397",    // 9 Taker buy base asset volume
#     "28.46694368",      // 10 Taker buy quote asset volume
#     "17928899.62484339" // 11 Ignore.
# ]

In [ ]:
klines = [
    [
        1650351360000,
        "3.25100000",
        "3.25213000",
        "3.23643000",
        "3.23725000",
        "136298.00000000",
        1650351419999,
        "442179.08394800",
        783,
        "37671.50000000",
        "122218.22250800",
        "0",
    ],
    [
        1650372900000,
        "3.29000000",
        "3.30000000",
        "3.29000000",
        "3.29532000",
        "182334.60000000",
        1650372959999,
        "600759.29294500",
        681,
        "162474.80000000",
        "535306.09749100",
        "0",
    ],
]

In [86]:
# Method for getting price
def get_price(ls):
    avg = (float(ls[1]) + float(ls[4])) / 2
    price = float("{:.4f}".format(avg))
    return price


# Method for getting percent change in 6 hour
def get_percent_change(ls_1, ls_2):
    price_1 = get_price(ls_1)
    price_2 = get_price(ls_2)

    change = (price_2 - price_1) / price_1

    change_in_percent = float("{:.4f}".format(change)) * 100

    return change_in_percent

In [111]:
from datetime import datetime, timedelta

klines = client.get_historical_klines(
    "GMTUSDT",
    Client.KLINE_INTERVAL_1MINUTE,
    str(datetime.now() - timedelta(hours=7)),
    str(datetime.now()),
)
# print(get_percent_change(klines[0],klines[-1]))
for lines in klines:
    print(lines[0], get_price(lines))
    print("\n")

1650391980000 3.7745


1650392040000 3.7652


1650392100000 3.7677


1650392160000 3.7699


1650392220000 3.7668


1650392280000 3.7657


1650392340000 3.7671


1650392400000 3.7694


1650392460000 3.7741


1650392520000 3.7728


1650392580000 3.7671


1650392640000 3.7652


1650392700000 3.7682


1650392760000 3.769


1650392820000 3.7625


1650392880000 3.7569


1650392940000 3.7576


1650393000000 3.7589


1650393060000 3.7485


1650393120000 3.7405


1650393180000 3.7376


1650393240000 3.735


1650393300000 3.7265


1650393360000 3.7163


1650393420000 3.7211


1650393480000 3.7266


1650393540000 3.7309


1650393600000 3.738


1650393660000 3.7413


1650393720000 3.7378


1650393780000 3.7381


1650393840000 3.7404


1650393900000 3.7378


1650393960000 3.7417


1650394020000 3.748


1650394080000 3.7473


1650394140000 3.747


1650394200000 3.7463


1650394260000 3.7458


1650394320000 3.7436


1650394380000 3.7426


1650394440000 3.7563


1650394500000 3.7642


1650394560000 3.

In [ ]:
# The following code was in code cell no 2
# Creating Socket Manager
bsm = BinanceSocketManager(client)

# Creating a socket for BTC-USDT pair
# socket = bsm.trade_socket('BTCUSDT')
socket = bsm.trade_socket(PAIR)

### Function for creating dataframe

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ["s", "E", "p"]]
    df.columns = ["symbol", "Time", "Price"]
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit="ms")
    return df

In [ ]:
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    while msg["e"] == "error":
        msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql("SLPUSDT", engine, if_exists="append", index=False)
    print(frame)

In [ ]:
msg = await socket.recv()
if msg["e"]:
    print(msg["e"])
print(msg)